In [27]:
import pandas as pd
import numpy as np
import re

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

import pickle

In [2]:
df = pd.read_csv("./processed_data.csv")
df.head()

,text,label
0,feel rather rotten ambitious right,sadness
1,update blog feel shitty,sadness
2,never make separate ever want feel like ashamed,sadness
3,leave bouquet red yellow tulips arm feel sligh...,joy
4,feel little vain one,sadness


In [3]:
#Spliting data values and target values
x = df['text']
y = df['label']

In [4]:
X_train, X_test, y_train, y_test = train_test_split (x,y, test_size=0.33, random_state=1)

In [5]:
# Create a CountVectorizer to convert text into a numerical representation and removing stop words
vectorizer = CountVectorizer(stop_words='english')

# Fit and transform the training data
X_train_vectorized = vectorizer.fit_transform(X_train)

# Transform the testing data
X_test_vectorized = vectorizer.transform(X_test)

In [30]:
# Dumping vectorizer as Pickle

with open('../../App/vectorizer.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)

In [6]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import f1_score

### Checking for all the classifiers

In [7]:
classifiers = [
    KNeighborsClassifier(15),
    SVC(),
    DecisionTreeClassifier(),
    ExtraTreesClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GaussianNB(),
    SGDClassifier(),
    MultinomialNB()
]

In [8]:
# for clf in classifiers:
#     clf.fit(X_train_vectorized, y_train)
#     pred=clf.predict(X_test_vectorized)
#     print(clf, f1_score(y_test, pred,average='weighted'))

### Extra Trees Classifier - 0.877

In [9]:
etc = ExtraTreesClassifier()
etc.fit(X_train_vectorized, y_train)
y_pred = etc.predict(X_test_vectorized)
accuracy = f1_score(y_test, y_pred,average='weighted')
print("Accuracy:", accuracy)

Accuracy: 0.8727944614076785


In [10]:
# etc = ExtraTreesClassifier(min_samples_split=5)
# etc.fit(X_train_vectorized, y_train)
# y_pred = etc.predict(X_test_vectorized)
# accuracy = f1_score(y_test, y_pred,average='weighted')
# print("Accuracy:", accuracy)

In [31]:
with open('../../App/model.pkl', 'wb') as file:
    pickle.dump(etc, file)

#### Hyperparameter Tuning

In [11]:
# from sklearn.model_selection import GridSearchCV

# etc = ExtraTreesClassifier()

# param_grid = {
#     'n_estimators': [50, 100, 150],
#     'max_depth': [None, 10, 20, 30],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4]
# }

# grid_search = GridSearchCV(estimator=etc, param_grid=param_grid, cv=5, scoring='f1_weighted')

# grid_search.fit(X_train_vectorized, y_train)

# print("Best Parameters:", grid_search.best_params_)

# best_etc = grid_search.best_estimator_

# y_pred = best_etc.predict(X_test_vectorized)

# accuracy = f1_score(y_test, y_pred, average='weighted')
# print("Best Model Accuracy:", accuracy)

In [12]:
# from sklearn.model_selection import RandomizedSearchCV
# from scipy.stats import randint

# etc = ExtraTreesClassifier()

# param_dist = {
#     'n_estimators': randint(50, 200),
#     'max_depth': [None] + list(np.random.randint(1, 50, 10)),
#     'min_samples_split': randint(2, 20),
#     'min_samples_leaf': randint(1, 20),
# }

# random_search = RandomizedSearchCV(estimator=etc, param_distributions=param_dist, n_iter=10, cv=5, scoring='f1_weighted', random_state=42)

# random_search.fit(X_train_vectorized, y_train)

# print("Best Parameters:", random_search.best_params_)

# best_etc = random_search.best_estimator_

# y_pred = best_etc.predict(X_test_vectorized)

# accuracy = f1_score(y_test, y_pred, average='weighted')
# print("Best Model Accuracy:", accuracy)

#### Predicting

In [13]:
new_text_entry = "Im having a very good time!! but i think the overall experience is slightly tragic, dont you think?"

new_text_entry_vectorized = vectorizer.transform([new_text_entry])

predicted_class = etc.predict(new_text_entry_vectorized)

print("Predicted Class:", predicted_class)

Predicted Class: ['sadness']


In [14]:
new_text_entry = "Today has been a long day. A little tiring but full of surprises. I got my first acceptance today. I was on cloud nine!!! But I did not get place to sit in the train. but thats only a minor inconvienence. Im less stressed now."

new_text_entry_vectorized = vectorizer.transform([new_text_entry])

predicted_class = etc.predict(new_text_entry_vectorized)

print("Predicted Class:", predicted_class)

Predicted Class: ['love']


In [26]:
input_text = "i am feeling pretty wonderful."
sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', input_text)

# Create a dictionary to store emotion counts
emotion_count = {'anger': 0, 'fear': 0, 'sadness': 0, 'love': 0, 'joy': 0 , 'surprise':0}  # Replace with actual emotion labels

# Process each sentence
for sentence in sentences:
    # Preprocess the sentence (optional, depending on your data)
    # processed_sentence = preprocess_function(sentence)

    # Transform the sentence
    sentence_vectorized = vectorizer.transform([sentence])

    # Predict emotion for the sentence
    emotion_prediction = etc.predict(sentence_vectorized)[0]

    # Update the emotion count
    emotion_count[emotion_prediction] += 1
    print(sentence, emotion_prediction)

# Print the results
for emotion, count in emotion_count.items():
    print(f"{emotion}: {count}")

i just feel too overwhelmed i can t see the forest for the trees as the saying goes. sadness
i am feeling pretty wonderful. joy
 anger
anger: 1
fear: 0
sadness: 1
love: 0
joy: 1
surprise: 0


### Random Forest Classifier - 0.872

In [16]:
rfc = RandomForestClassifier()
rfc.fit(X_train_vectorized, y_train)
y_pred = rfc.predict(X_test_vectorized)
accuracy = f1_score(y_test, y_pred,average='weighted')
print("Accuracy:", accuracy)

Accuracy: 0.8627248765439679


In [17]:
rfc = RandomForestClassifier(min_samples_split=15)
rfc.fit(X_train_vectorized, y_train)
y_pred = rfc.predict(X_test_vectorized)
accuracy = f1_score(y_test, y_pred,average='weighted')
print("Accuracy:", accuracy)

Accuracy: 0.869391076292237


### SGD Classifier - 0.8758

In [18]:
sgd = SGDClassifier()
sgd.fit(X_train_vectorized, y_train)
y_pred = sgd.predict(X_test_vectorized)
accuracy = f1_score(y_test, y_pred,average='weighted')
print("Accuracy:", accuracy)

Accuracy: 0.869284125810479


In [20]:
# sgd_clf = SGDClassifier()

# param_grid = {
#     'loss': ['hinge', 'log', 'modified_huber'],
#     'penalty': ['l1', 'l2', 'elasticnet'],
#     'alpha': [0.0001, 0.001, 0.01, 0.1],
#     'max_iter': [1000, 2000, 3000],
#     'tol': [1e-3, 1e-4, 1e-5]
# }

# grid_search = GridSearchCV(estimator=sgd_clf, param_grid=param_grid, cv=5, scoring='f1_weighted')

# grid_search.fit(X_train_vectorized, y_train)

# print("Best Parameters:", grid_search.best_params_)

# best_sgd_clf = grid_search.best_estimator_

# y_pred = best_sgd_clf.predict(X_test_vectorized)

# accuracy = f1_score(y_test, y_pred, average='weighted')
# print("Best Model Accuracy:", accuracy)